In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
train_transform = transforms.Compose([
    # Random augmentations (different each time!) 
    transforms .RandomHorizontalFlip(p=0.5), 
    transforms .RandomRotation(degrees=10), 
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

In [4]:
train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=train_transform
)

test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=val_transform
)

100%|██████████| 170M/170M [00:02<00:00, 83.5MB/s]


In [5]:
train_loader =  DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4, #Permite que dois núcleos da CPU preparem os dados simultaneamente.
    pin_memory=True
               
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4, #Permite que dois núcleos da CPU preparem os dados simultaneamente.
    pin_memory=True
)

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3,
                      stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3,
                      stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

        #Downsaple para ajustar shape
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels,
                          kernel_size=1, stride=stride, bias=False), 
                nn.BatchNorm2d(out_channels)
            )

    def _initial_forward(self, x):
        out = self.conv_block_1(x)
        out = self.conv_block_2(out)
        return out
          
    def forward(self, x):
        identity = x
    
        out = self._initial_forward(x)
    
        if self.downsample is not None:
            identity = self.downsample(x)
    
        out = out + identity
        out = F.relu(out)
    
        return out

In [7]:
class SimpleResNet(nn.Module):
    def __init__(self,):
        super().__init__()

        self.initial_block = nn.Sequential(
            nn.LazyConv2d(16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )

        self.res_block1 = ResidualBlock(16, 16)
        self.res_block2 = ResidualBlock(16, 32, stride=2)#downsampling
        self.res_block3 = ResidualBlock(32, 64, stride=2)

        self.final_block = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.LazyLinear(10)
        )

    def forward(self, x):
        x = self.initial_block(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.final_block(x)
        return x

In [8]:
model = SimpleResNet().to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

In [9]:
print("Iniciando o treinamento...")
model.train()

for epoch in range(51):
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    if epoch % 10 == 0:
        print("training epoch: ", epoch)
        print("loss:", running_loss/len(train_loader))


Iniciando o treinamento...
training epoch:  0
loss: 1.5024921573946237
training epoch:  10
loss: 0.6875979270014312
training epoch:  20
loss: 0.5881996294650276
training epoch:  30
loss: 0.5507281173373122
training epoch:  40
loss: 0.5264122638174945
training epoch:  50
loss: 0.5078610979839969
